In [ ]:
import keras,os
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.losses import MeanSquaredError

In [ ]:
import dlmodels

In [ ]:
################sar
sentinel_1 = dlmodels.complex_vgg()
sentinel_2 = dlmodels.vgg16()
GEDI = dlmodels.vgg16_gedi()

In [ ]:
def fusion(sentinel_1,sentinel_2,GEDI):
    
    merge = layers.concatenate([sentinel_1,sentinel_2,GEDI])
    
    # fusion
    x = Conv2D(filters=128, kernel_size=(3, 3), padding='same', activation='relu')(merge)
    x = Conv2D(filters=128, kernel_size=(3, 3), padding='same', activation='relu')(x)
    
    return x
    

In [ ]:
fusion_feature = fusion(sentinel_1,sentinel_2,GEDI)

In [ ]:
def task(input_shape):
    inputs = layers.Input(input_shape)
    x = Conv2D(filters=128, kernel_size=(3, 3), padding='same', activation='relu')(inputs)
    x = Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu')(x)
    x = Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu')(x)
    output_250 = Conv2D(filters=1, kernel_size=(3, 3), padding='same', activation='linear')(x)

   
    
    ######upsample###
    y= upsample()(inputs)
    output_100 = Conv2D(filters=1, kernel_size=(3, 3), padding='same', activation='linear')(y)
    
    
    model = output.Model(inputs=inputs, outputs=[output_250, output_100])
    
    return model


In [ ]:
output = task(fusion_features)
losses = {
    'output_250': 'mean_squared_error',
    'output_100': 'mean_squared_error'
}

# 设置损失权重
loss_weights = {
    'output_250': 0.5,  # task250的损失权重
    'output_100': 0.5   # task100的损失权重
}

output.compile(optimizer='adam', loss=losses, loss_weights=loss_weights)
#output.fit([sar,optical,gedi],[out250,out100])